# Reproduce Allen smFISH results with Starfish

The `allen_smFISH.zip` file needed to follow along with this notebook can be downloaded [here]()

This notebook walks through a work flow that reproduces the smFISH result for one field of view using the starfish package. 
It assumes that you have unzipped `allen_smFISH.zip` in the same directory as this notebook. Thus, you should see:

raw/
allen_smFISH.ipynb

In [1]:
import os
from starfish.io import Stack
import numpy as np

from starfish.pipeline.filter.clip import Clip
from starfish.pipeline.filter.bandpass import Bandpass
from starfish.pipeline.filter.gaussian_low_pass import GaussianLowPass

In [2]:
%matplotlib inline

from scipy import ndimage as ndi
from scipy import stats

from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util, img_as_float)

In [3]:
# package this up same as before
experiment_json = os.path.expanduser('~/google_drive/starfish/data/allen_smFISH/experiment.json')

In [4]:
s = Stack()
s.read(experiment_json)

In [5]:
# The allen's data is uint, but starfish loads it as floats. Cast it to int here, which does not cause any loss of precision. 
s.image._data = s.image._data.astype(np.uint16)
istack = s.image

In [6]:
s_clip = Clip(p_min=10, p_max=100)
s_clip.filter(s)

In [23]:
s_bandpass = Bandpass(lshort=0.5, llong=7, threshold=None, truncate=4)
s_bandpass.filter(s)

In [ ]:
glp = Gauss